In [ ]:
SHAP_ENABLED=False

In [ ]:
import os
import pandas as pd
import joblib
import xgboost as xgb
import gc
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from xgboost import XGBClassifier

if SHAP_ENABLED:
    import shap

In [ ]:
import platform

SHOULD_GENERATE_IMAGES = platform.system() == 'Windows'
if "SHOULD_GENERATE_IMAGES" in os.environ:
    SHOULD_GENERATE_IMAGES = True

In [ ]:
IS_HYPER_PARAMETER_SEARCH = False
NUM_FOLDS_CROSSVALIDATION = 10
NUM_TREES = 120
TREE_DEPTH = 5

DATA_OUTPUT_DIR = os.path.join('..','00.data','output')
DATA_HYPERPARAMETERS_DIR = os.path.join('..','00.data','hyperparameters')

In [ ]:
if "IS_HYPER_PARAMETER_SEARCH" in os.environ:
    IS_HYPER_PARAMETER_SEARCH = bool(os.environ["IS_HYPER_PARAMETER_SEARCH"])
    
if "NUM_FOLDS_CROSSVALIDATION" in os.environ:
    NUM_FOLDS_CROSSVALIDATION = int(os.environ["NUM_FOLDS_CROSSVALIDATION"])
    
if "DATA_OUTPUT_DIR" in os.environ:
    DATA_OUTPUT_DIR = os.environ["DATA_OUTPUT_DIR"]
    
if "DATA_HYPERPARAMETERS_DIR" in os.environ:
    DATA_HYPERPARAMETERS_DIR = os.environ["DATA_HYPERPARAMETERS_DIR"]    
    
if "NUM_TREES" in os.environ:
    NUM_TREES = os.environ["NUM_TREES"]

if "TREE_DEPTH" in os.environ:
    TREE_DEPTH = os.environ["TREE_DEPTH"]

In [ ]:
files_found = [x for x in os.listdir(DATA_OUTPUT_DIR) if x.endswith('.pickle')]
files_found_tokens=[x.split('.') for x in files_found]

models_found = {}

for current_model in files_found_tokens:
    model_name = '.'.join(current_model[:-2])
    filename = os.path.join(DATA_OUTPUT_DIR,'.'.join(current_model))
    if not model_name in models_found:
        models_found[model_name]= {}          
    models_found[model_name][current_model[-2]]=filename

print(models_found)

In [ ]:
first_model = list(models_found.keys())[0]
current_total_dataset = joblib.load(models_found[first_model]['total'])
current_parameters = joblib.load(models_found[first_model]['parameters'])

In [ ]:
df_current_total_dataset = pd.DataFrame(current_total_dataset)
df_current_total_dataset_x = df_current_total_dataset[current_parameters['CURRENT_X_COLUMNS']]
df_current_total_dataset_y_short = df_current_total_dataset[current_parameters['CURRENT_Y_COLUMN_SHORT']]
df_current_total_dataset_y_long = df_current_total_dataset[current_parameters['CURRENT_Y_COLUMN_LONG']]

total_total_count = len(df_current_total_dataset)
total_short_count = len([x for x in df_current_total_dataset_y_short if x == True])
total_long_count = len([x for x in df_current_total_dataset_y_long if x == True])
total_long_ratio = (total_total_count - total_long_count) / total_long_count
total_short_ratio = (total_total_count - total_short_count) / total_short_count

print(f'(train)Short Count:{total_short_count}/{total_total_count} {(total_short_count/total_total_count) * 100:.2f}%')
print(f'(train)Long Count:{total_long_count}/{total_total_count}  {(total_long_count/total_total_count) * 100:.2f}%')
print(f'(train)Long Ratio:{total_long_ratio:.2f}')
print(f'(train)Short Ratio:{total_short_ratio:.2f}')

In [ ]:
CURRENT_EXCHANGE = current_parameters['CURRENT_EXCHANGE']
CURRENT_ASSET = current_parameters['CURRENT_ASSET']
CURRENT_TIMEFRAME = current_parameters['CURRENT_TIMEFRAME']
CURRENT_TARGET = current_parameters['CURRENT_TARGET']
CURRENT_STOP = current_parameters['CURRENT_STOP']
MINIMUM_TIME = current_parameters['MINIMUM_TIME']
MAXIMUM_TIME = current_parameters['MAXIMUM_TIME']
MAX_TRAIN_DATE = current_parameters['MAX_TRAIN_DATE']
MAX_TRADE_DURATION = current_parameters['MAX_TRADE_DURATION']
CURRENT_DAILY_FILE_CSV = current_parameters['CURRENT_DAILY_FILE_CSV']
CURRENT_5MIN_FILE_CSV = current_parameters['CURRENT_5MIN_FILE_CSV']
MINIMUM_DATE_DATAFRAME = current_parameters['MINIMUM_DATE_DATAFRAME']
MINIMUM_DATE_TRADE = current_parameters['MINIMUM_DATE_TRADE']
DECISION_BOUNDARY = current_parameters['DECISION_BOUNDARY']
TRADE_DAY_START = current_parameters['TRADE_DAY_START']
WEIGHT_RATIO = float(current_parameters['WEIGHT_RATIO'])

In [ ]:
FULL_FEATURE_LIST = list(
    {'x0_roc', 'x0_previous_high', 'x0_previous_low', 'x0_previous_close', 'x0_previous_open', 'x0_ema9_close',
     'x0_ema21_close', 'x0_ema55_close', 'x0_ema144_close', 'x0_ema233_close', 'x1_roc', 'x1_previous_high',
     'x1_previous_low', 'x1_previous_close', 'x1_previous_open', 'x1_ema9_close', 'x1_ema21_close', 'x1_ema55_close',
     'x1_ema144_close', 'x1_ema233_close', 'x1_x0_close', 'x1_x0_high', 'x1_x0_low', 'x2_roc', 'x2_previous_high',
     'x2_previous_low', 'x2_previous_close', 'x2_previous_open', 'x2_ema9_close', 'x2_ema21_close', 'x2_ema55_close',
     'x2_ema144_close', 'x2_ema233_close', 'x2_x1_close', 'x2_x1_high', 'x2_x1_low', 'x3_roc', 'x3_previous_high',
     'x3_previous_low', 'x3_previous_close', 'x3_previous_open', 'x3_ema9_close', 'x3_ema21_close', 'x3_ema55_close',
     'x3_ema144_close', 'x3_ema233_close', 'x3_x2_close', 'x3_x2_high', 'x3_x2_low', 'x4_roc', 'x4_previous_high',
     'x4_previous_low', 'x4_previous_close', 'x4_previous_open', 'x4_ema9_close', 'x4_ema21_close', 'x4_ema55_close',
     'x4_ema144_close', 'x4_ema233_close', 'x4_x3_close', 'x4_x3_high', 'x4_x3_low', 'x5_roc', 'x5_previous_high',
     'x5_previous_low', 'x5_previous_close', 'x5_previous_open', 'x5_ema9_close', 'x5_ema21_close', 'x5_ema55_close',
     'x5_ema144_close', 'x5_ema233_close', 'x5_x4_close', 'x5_x4_high', 'x5_x4_low', 'x6_roc', 'x6_previous_high',
     'x6_previous_low', 'x6_previous_close', 'x6_previous_open', 'x6_ema9_close', 'x6_ema21_close', 'x6_ema55_close',
     'x6_ema144_close', 'x6_ema233_close', 'x6_x5_close', 'x6_x5_high', 'x6_x5_low', 'x7_ema9', 'x7_ema21', 'x7_ema55',
     'x7_ema144', 'x7_ema233', 'x7_ema9', 'x7_ema21', 'x7_ema55', 'x7_ema144', 'x7_ema233', 'x7_open', 'x7_high',
     'x7_low', 'x7_close', 'x7_volume', 'x7_open', 'x7_high', 'x7_low', 'x7_close', 'x7_height', 'x7_body', 'x7_roc',
     'x7_previous_high', 'x7_previous_low', 'x7_previous_close', 'x7_previous_open', 'x7_ema9_close', 'x7_ema21_close',
     'x7_ema55_close', 'x7_ema144_close', 'x7_ema233_close', 'x7_x6_close', 'x7_x6_high', 'x7_x6_low', 'x8_roc',
     'x8_previous_high', 'x8_previous_low', 'x8_previous_close', 'x8_previous_open', 'x8_ema9_close', 'x8_ema21_close',
     'x8_ema55_close', 'x8_ema144_close', 'x8_ema233_close', 'x8_x7_close', 'x8_x7_high', 'x8_x7_low', 'x9_roc',
     'x9_previous_high', 'x9_previous_low', 'x9_previous_close', 'x9_previous_open', 'x9_ema9_close', 'x9_ema21_close',
     'x9_ema55_close', 'x9_ema144_close', 'x9_ema233_close', 'x9_x8_close', 'x9_x8_high', 'x9_x8_low', 'x10_roc',
     'x10_previous_high', 'x10_previous_low', 'x10_previous_close', 'x10_previous_open', 'x10_ema9_close',
     'x10_ema21_close', 'x10_ema55_close', 'x10_ema144_close', 'x10_ema233_close', 'x10_x9_close', 'x10_x9_high',
     'x10_x9_low', 'x11_roc', 'x11_previous_high', 'x11_previous_low', 'x11_previous_close', 'x11_previous_open',
     'x11_ema9_close', 'x11_ema21_close', 'x11_ema55_close', 'x11_ema144_close', 'x11_ema233_close', 'x11_x10_close',
     'x11_x10_high', 'x11_x10_low', 'previous_close', 'previous_high', 'previous_low', 'previous_open', 'x0_ema55',
     'x0_ema144', 'x0_ema233', 'x1_ema55', 'x1_ema144', 'x1_ema233', 'x2_ema55', 'x2_ema144', 'x2_ema233', 'x3_ema55',
     'x3_ema144', 'x3_ema233', 'x4_ema55', 'x4_ema144', 'x4_ema233', 'x5_ema55', 'x5_ema144', 'x5_ema233', 'x6_ema55',
     'x6_ema144', 'x6_ema233', 'x7_ema55', 'x7_ema144', 'x7_ema233', 'x8_ema55', 'x8_ema144', 'x8_ema233', 'x9_ema55',
     'x9_ema144', 'x9_ema233', 'x10_ema55', 'x10_ema144', 'x10_ema233', 'x11_ema55', 'x11_ema144', 'x11_ema233',
     'x0_body', 'x0_close', 'x0_ema21', 'x0_ema9', 'x0_height', 'x0_high', 'x0_low', 'x0_open', 'x0_volume', 'x10_body',
     'x10_close', 'x10_ema21', 'x10_ema9', 'x10_height', 'x10_high', 'x10_low', 'x10_open', 'x10_volume', 'x11_body',
     'x11_close', 'x11_close_slope', 'x11_ema21', 'x11_ema9', 'x11_height', 'x11_high', 'x11_high_slope', 'x11_low',
     'x11_low_slope', 'x11_open', 'x11_volume', 'x11_volume_slope', 'x1_body', 'x1_close', 'x1_ema21', 'x1_ema9',
     'x1_height', 'x1_high', 'x1_low', 'x1_open', 'x1_volume', 'x2_body', 'x2_close', 'x2_ema21', 'x2_ema9',
     'x2_height', 'x2_high', 'x2_low', 'x2_open', 'x2_volume', 'x3_body', 'x3_close', 'x3_ema21', 'x3_ema9',
     'x3_height', 'x3_high', 'x3_low', 'x3_open', 'x3_volume', 'x4_body', 'x4_close', 'x4_ema21', 'x4_ema9',
     'x4_height', 'x4_high', 'x4_low', 'x4_open', 'x4_volume', 'x5_body', 'x5_close', 'x5_ema21', 'x5_ema9',
     'x5_height', 'x5_high', 'x5_low', 'x5_open', 'x5_volume', 'x6_body', 'x6_close', 'x6_ema21', 'x6_ema9',
     'x6_height', 'x6_high', 'x6_low', 'x6_open', 'x6_volume', 'x7_body', 'x7_close', 'x7_ema21', 'x7_ema9',
     'x7_height', 'x7_high', 'x7_low', 'x7_open', 'x7_volume', 'x8_body', 'x8_close', 'x8_ema21', 'x8_ema9',
     'x8_height', 'x8_high', 'x8_low', 'x8_open', 'x8_volume', 'x9_body', 'x9_close', 'x9_ema21', 'x9_ema9',
     'x9_height', 'x9_high', 'x9_low', 'x9_open', 'x9_volume', 'current_bar_in_date', "x0_vwap", "x1_vwap", "x2_vwap",
     "x3_vwap", "x4_vwap", "x5_vwap", "x6_vwap", "x7_vwap", "x8_vwap", "x9_vwap", "x10_vwap", "x11_vwap"})

In [ ]:
SHORT_FEATURE_LIST = ['previous_close', 'previous_high', 'previous_low', 'previous_open', 'x0_ema233_close', 'x0_height', 'x0_previous_close', 
                      'x0_previous_high', 'x0_previous_low', 'x0_previous_open', 'x0_volume', 'x0_vwap', 'x10_ema233_close', 'x10_height', 
                      'x10_previous_close', 'x10_previous_high', 'x10_previous_low', 'x10_previous_open', 'x10_volume', 'x10_vwap', 'x11_ema233_close', 
                      'x11_height', 'x11_previous_close', 'x11_previous_high', 'x11_previous_low', 'x11_previous_open', 'x11_volume', 'x11_vwap', 
                      'x5_ema233_close', 'x5_height', 'x5_previous_close', 'x5_previous_high', 'x5_previous_low', 'x5_previous_open', 'x5_volume', 'x5_vwap']

In [ ]:
parameter_file_name = test_output_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.parameters.pickle")
parameter_full_file_name = os.path.join(DATA_OUTPUT_DIR,test_output_file_name)
parameters = {
    "CURRENT_EXCHANGE": CURRENT_EXCHANGE,
    "TRADE_DAY_START": TRADE_DAY_START,
    "MINIMUM_TIME" : MINIMUM_TIME,
    "MAXIMUM_TIME" : MAXIMUM_TIME,
    "MINIMUM_DATE_DATAFRAME" : MINIMUM_DATE_DATAFRAME,
    "MINIMUM_DATE_TRADE" : MINIMUM_DATE_TRADE,
    "MAX_TRAIN_DATE" : MAX_TRAIN_DATE,
    "MAX_TRADE_DURATION" : MAX_TRADE_DURATION,
    "CURRENT_TARGET" : CURRENT_TARGET,
    "CURRENT_STOP" : CURRENT_STOP,
    "CURRENT_ASSET" : CURRENT_ASSET,
    "CURRENT_TIMEFRAME" : CURRENT_TIMEFRAME,
    "CURRENT_DAILY_FILE_CSV" : CURRENT_DAILY_FILE_CSV,
    "CURRENT_5MIN_FILE_CSV" : CURRENT_5MIN_FILE_CSV,
    "DECISION_BOUNDARY": DECISION_BOUNDARY,
    "NUM_TREES": NUM_TREES,
    "TREE_DEPTH": TREE_DEPTH,
    "WEIGHT_RATIO": WEIGHT_RATIO,
    "CURRENT_Y_COLUMN_SHORT" : 'is_short',
    "CURRENT_Y_COLUMN_LONG" : 'is_long',
    "CURRENT_X_COLUMNS" : FULL_FEATURE_LIST
}

joblib.dump(parameters, parameter_full_file_name, True)

In [ ]:
if IS_HYPER_PARAMETER_SEARCH:
    folds = NUM_FOLDS_CROSSVALIDATION
    
    params_grid_search = {
            'objective' : ['binary:logitraw'],
            'eval_metric' : ['auc'],
            'tree_method' : ['exact'],
            'max_depth': [5],
            'gamma': [1],
            'eta': [0.3],
            'n_estimators' : list(range(50,200,5)),
            'scale_pos_weight' : [total_short_ratio]
    }
    
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

    grid_search_short = GridSearchCV(XGBClassifier(use_label_encoder=False), 
                                       param_grid=params_grid_search, 
                                       scoring='roc_auc', 
                                       n_jobs=-1, cv=skf.split(df_current_total_dataset_x,df_current_total_dataset_y_short), 
                                       verbose=3 )

    grid_search_short=grid_search_short.fit(df_current_total_dataset_x, df_current_total_dataset_y_short)        

    print('Short Best Params:')
    print('------------------')
    print(grid_search_short.best_params_)
    print('grid_search_short Best Estimator:')
    print('------------------')
    print(grid_search_short.best_estimator_)

    best_short_booster = grid_search_short.best_estimator_.get_booster()
    
    hyperparam_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.hyperparameters_short.xlsx")
    hyperparam_full_file_name = os.path.join(DATA_OUTPUT_DIR,hyperparam_file_name)

    results = pd.DataFrame(grid_search_short.cv_results_)
    results.to_excel(hyperparam_full_file_name, index=False)    

In [ ]:
if not IS_HYPER_PARAMETER_SEARCH:
        
    best_short_booster = xgb.XGBClassifier(objective="binary:logitraw", 
                                  eval_metric="auc",
                                  tree_method="exact",
                                  max_depth=int(TREE_DEPTH),
                                  gamma=1,
                                  eta=0.3,
                                  subsample=1.0,
                                  colsample_bytree=1.0,
                                  colsample_bylevel=1.0,
                                  colsample_bynode=1.0,
                                  use_label_encoder=False,
                                  silent=False,                                           
                                  n_estimators=int(NUM_TREES),
                                  scale_pos_weight=total_short_ratio * WEIGHT_RATIO)
    
    best_short_booster.fit(df_current_total_dataset_x, df_current_total_dataset_y_short.to_numpy())         
    
    best_short_booster_score = best_short_booster.score(df_current_total_dataset_x, df_current_total_dataset_y_short.to_numpy())
    

In [ ]:
if IS_HYPER_PARAMETER_SEARCH:
    folds = NUM_FOLDS_CROSSVALIDATION

    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

    grid_search_long = GridSearchCV(XGBClassifier(use_label_encoder=False), 
                                       param_grid=params_grid_search, 
                                       scoring='roc_auc', 
                                       n_jobs=-1, cv=skf.split(df_current_total_dataset_x,df_current_total_dataset_y_long), 
                                       verbose=3 )

    grid_search_long = grid_search_long.fit(df_current_total_dataset_x, df_current_total_dataset_y_long)
    
    print('Long Best Params:')
    print('------------------')
    print(grid_search_long.best_params_)
    print('Long Best Estimator:')
    print('------------------')
    print(grid_search_long.best_estimator_)

    best_long_booster = grid_search_long.best_estimator_.get_booster()   
    
    hyperparam_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.hyperparameters_long.xlsx")
    hyperparam_full_file_name = os.path.join(DATA_OUTPUT_DIR,hyperparam_file_name)

    results = pd.DataFrame(grid_search_long.cv_results_)
    results.to_excel(hyperparam_full_file_name, index=False)    

In [ ]:
if not IS_HYPER_PARAMETER_SEARCH:
        
    best_long_booster = xgb.XGBClassifier(objective="binary:logitraw", 
                                  eval_metric="auc",
                                  tree_method="exact",
                                  max_depth=int(TREE_DEPTH),
                                  gamma=1,
                                  eta=0.3,
                                  subsample=1.0,
                                  colsample_bytree=1.0,
                                  colsample_bylevel=1.0,
                                  colsample_bynode=1.0,
                                  use_label_encoder=False,
                                  n_estimators=int(NUM_TREES),
                                  silent=False,
                                  scale_pos_weight=total_long_ratio * WEIGHT_RATIO)
    
    best_long_booster.fit(df_current_total_dataset_x, df_current_total_dataset_y_long.to_numpy())  
    
    best_long_booster_score = best_long_booster.score(df_current_total_dataset_x, df_current_total_dataset_y_long.to_numpy())

In [ ]:
feature_importance = best_short_booster.get_booster().get_score(importance_type='weight')

features = []
for current_key in list(feature_importance.keys()):
    features.append({ 'feature': current_key, 'value' : feature_importance[current_key]})

features.sort(key=lambda x:x['value'], reverse=True)    
    
data = pd.DataFrame(features)
feature_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.hyperparameters_short.xlsx")
feature_full_file_name = os.path.join(DATA_OUTPUT_DIR,feature_file_name)
data.to_excel(feature_full_file_name, index=False)    

data.head(20)

In [ ]:
feature_importance = best_long_booster.get_booster().get_score(importance_type='weight')

features = []
for current_key in list(feature_importance.keys()):
    features.append({ 'feature': current_key, 'value' : feature_importance[current_key]})

features.sort(key=lambda x:x['value'], reverse=True)    
    
data = pd.DataFrame(features)
feature_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.hyperparameters_long.xlsx")
feature_full_file_name = os.path.join(DATA_OUTPUT_DIR,feature_file_name)
data.to_excel(feature_full_file_name, index=False)    

data.head(20)

In [ ]:
if SHAP_ENABLED:
    shap.initjs()
    explainer = shap.TreeExplainer(best_long_booster)
    shap_values = explainer.shap_values(df_current_total_dataset_x)
    shap.force_plot(explainer.expected_value, shap_values[0,:], df_current_total_dataset_x.iloc[0,:])

In [ ]:
if SHAP_ENABLED:
    shap.initjs()
    shap_values = shap.TreeExplainer(best_long_booster).shap_values(df_current_total_dataset_x)
    shap.summary_plot(shap_values, df_current_total_dataset_x, plot_type="bar")

In [ ]:
if SHAP_ENABLED:
    shap.initjs()
    explainer = shap.TreeExplainer(best_short_booster)
    shap_values = explainer.shap_values(df_current_total_dataset_x)
    shap.force_plot(explainer.expected_value, shap_values[0,:], df_current_total_dataset_x.iloc[0,:])

In [ ]:
if SHAP_ENABLED:
    shap.initjs()
    shap_values = shap.TreeExplainer(best_short_booster).shap_values(df_current_total_dataset_x)
    shap.summary_plot(shap_values, df_current_total_dataset_x, plot_type="bar")

In [ ]:
CURRENT_EXCHANGE = current_parameters['CURRENT_EXCHANGE']
CURRENT_ASSET = current_parameters['CURRENT_ASSET']
CURRENT_TIMEFRAME = current_parameters['CURRENT_TIMEFRAME']
CURRENT_TARGET = current_parameters['CURRENT_TARGET']
CURRENT_STOP = current_parameters['CURRENT_STOP']

In [ ]:
def predict_short(row):
    a = row[current_parameters['CURRENT_X_COLUMNS']].to_numpy().reshape(1,-1)
    return best_short_booster.get_booster().inplace_predict(a)[0]

def predict_long(row):
    a = row[current_parameters['CURRENT_X_COLUMNS']].to_numpy().reshape(1,-1)
    return best_long_booster.get_booster().inplace_predict(a)[0]

df_current_total_dataset['short_predict'] = df_current_total_dataset.apply( lambda row: predict_short(row), axis=1)
df_current_total_dataset['long_predict'] = df_current_total_dataset.apply( lambda row: predict_long(row), axis=1)


In [ ]:
current_total_dataset = None
dataset_matrix_short_test = None
dataset_matrix_long_test = None
dataset_matrix_short_train = None
dataset_matrix_long_train = None

df_current_test_dataset = None
current_test_dataset = None
df_current_test_dataset_x = None
df_current_test_dataset_y_short = None
df_current_test_dataset_y_long = None

df_current_train_dataset = None
current_train_dataset = None
df_current_train_dataset_x = None
df_current_train_dataset_y_short = None
df_current_train_dataset_y_long = None

gc.collect()

In [ ]:
total_output_file_name = f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.xlsx"
total_output_full_file_name = os.path.join(DATA_OUTPUT_DIR,total_output_file_name)

In [ ]:
df_current_total_dataset = None
gc.collect()

In [ ]:
model_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.xgboostshortmodel.txt")
model_full_file_name = os.path.join(DATA_OUTPUT_DIR,model_file_name)

best_short_booster.get_booster().dump_model(model_full_file_name)

In [ ]:
model_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.xgboostlongmodel.txt")
model_full_file_name = os.path.join(DATA_OUTPUT_DIR,model_file_name)

best_long_booster.get_booster().dump_model(model_full_file_name)

In [ ]:
model_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.xgboostshortmodel.pickle")
model_full_file_name = os.path.join(DATA_OUTPUT_DIR,model_file_name)

joblib.dump(best_short_booster, model_full_file_name, True)

In [ ]:
model_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.xgboostlongmodel.pickle")
model_full_file_name = os.path.join(DATA_OUTPUT_DIR,model_file_name)

joblib.dump(best_long_booster, model_full_file_name, True)

In [ ]:
model_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.xgboostlongmodel.pickle")

In [ ]:
long_score = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.long.train.score.txt")
long_score_file_name = os.path.join(DATA_OUTPUT_DIR,long_score)

short_score = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.short.train.score.txt")
short_score_file_name = os.path.join(DATA_OUTPUT_DIR,short_score)

with open(long_score_file_name,"w") as f:
    f.write(str(best_long_booster_score))

with open(short_score_file_name,"w") as f:
    f.write(str(best_short_booster_score))

In [ ]:
if IS_HYPER_PARAMETER_SEARCH:
    model_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.xgboostclassifiershortmodel.pickle")
    model_full_file_name = os.path.join(DATA_OUTPUT_DIR,model_file_name)

    joblib.dump(grid_search_short.best_estimator_, model_full_file_name, True)

In [ ]:
if IS_HYPER_PARAMETER_SEARCH:
    model_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.xgboostclassifierlongmodel.pickle")
    model_full_file_name = os.path.join(DATA_OUTPUT_DIR,model_file_name)

    joblib.dump(grid_search_long.best_estimator_, model_full_file_name, True)